In [1]:
import sys
print(sys.executable)

c:\users\krishna\appdata\local\programs\python\python39\python.exe


In [5]:
import tensorflow as tf
print(tf.__version__)


2.18.0


In [2]:
import win32api
print(win32api.GetVersion())


1483145226


In [6]:
import numpy
print(numpy.__version__)


2.0.2


In [4]:
import debugpy
print(debugpy.__version__)


1.8.8


In [8]:
import pandas as pd 
import numpy as np 
import os
from glob import glob
from PIL import Image
import shutil
from tqdm import tqdm
import itertools
import matplotlib.pyplot as plt
import cv2
import warnings
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from keras.utils import to_categorical
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [9]:
# Suppress warnings
warnings.filterwarnings('ignore')


In [10]:
# Read metadata
skinDf = pd.read_csv("D:/SKIN CANCER CLASSIFICATION/HAM10000_metadata.csv")
# Set image size
img_size = (64, 64)

In [11]:
# Encode labels
labelEncoder = LabelEncoder()
skinDf['label'] = labelEncoder.fit_transform(skinDf['dx'])

# Resample to balance classes
dfs_by_label_resampled = {}
n_samples = 800
for label in range(7):
    df_label_resampled = resample(skinDf[skinDf['label'] == label], n_samples=n_samples, replace=True, random_state=42)
    dfs_by_label_resampled[label] = df_label_resampled
balanced_df = pd.concat(dfs_by_label_resampled.values()).sample(frac=1, random_state=42).reset_index(drop=True)


In [20]:
import os

if not os.path.exists('D:/SKIN CANCER CLASSIFICATION/images'):
    print("Directory does not exist")
else:
    print("Directory found")


Directory found


In [26]:
# Load available image files into a set
available_files = {os.path.splitext(os.path.basename(x))[0] for x in glob(os.path.join('D:/SKIN CANCER CLASSIFICATION/images', '*.jpg'))}

# Filter out rows where 'image_id' is not in the available files
balanced_df = balanced_df[balanced_df['image_id'].isin(available_files)].reset_index(drop=True)

# Check the updated dataframe
print(f"Number of entries after removing missing files: {len(balanced_df)}")




Number of entries after removing missing files: 2793


In [27]:
# Define the target image size
img_size = (224, 224)  # Example size; adjust as needed

# Load image paths from the provided directory
imgPath = {
    os.path.splitext(os.path.basename(x))[0]: x 
    for x in glob(os.path.join(r'D:\SKIN CANCER CLASSIFICATION\images', '*.jpg'))
}

# Function to safely load and process images
def load_image(file_path):
    if file_path and os.path.exists(file_path):  # Check if the file exists
        try:
            img = Image.open(file_path).convert("RGB")  # Convert to RGB to ensure consistency
            img_resized = img.resize(img_size)
            return np.asarray(img_resized) / 255.0  # Normalize pixel values to range [0, 1]
        except Exception as e:
            print(f"Error loading image {file_path}: {e}")
            return None
    else:
        print(f"Image file not found: {file_path}")
        return None

# Assuming you have a DataFrame `balanced_df` with an `image_id` column
balanced_df['image'] = balanced_df['image_id'].map(imgPath.get).map(load_image)

# Handle any missing or invalid images
balanced_df = balanced_df.dropna(subset=['image']).reset_index(drop=True)  # Drop rows with invalid images

# Convert image data and labels to arrays
x = np.stack(balanced_df['image'].to_list())  # Stack images into a single array
y = to_categorical(balanced_df['label'], num_classes=7)  # Convert labels to one-hot encoding

# Split the data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, shuffle=True)

# Verify data shapes
print(f"x_train shape: {x_train.shape}")
print(f"x_test shape: {x_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")


x_train shape: (2234, 224, 224, 3)
x_test shape: (559, 224, 224, 3)
y_train shape: (2234, 7)
y_test shape: (559, 7)


In [32]:
# Define model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', input_shape=(224, 224, 3)),  # Update input shape
    MaxPooling2D(),
    BatchNormalization(),
    
    Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal'),
    Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal'),
    MaxPooling2D(),
    BatchNormalization(),
    
    Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal'),
    Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal'),
    MaxPooling2D(),
    BatchNormalization(),
    
    Flatten(),
    Dropout(rate=0.5),
    Dense(256, activation='relu', kernel_initializer='he_normal'),
    Dropout(rate=0.5),
    Dense(128, activation='relu', kernel_initializer='he_normal'),
    Dropout(rate=0.5),
    Dense(32, activation='relu', kernel_initializer='he_normal'),
    BatchNormalization(),
    Dense(units=7, activation='softmax', kernel_initializer='glorot_uniform', name='classifier')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)
early_stopping = EarlyStopping(patience=5)

# Resize images to match model input shape
img_size = (224, 224)  # Resize images to 224x224

# Assuming you are loading images with a function like this:
def load_image(file_path):
    if file_path and os.path.exists(file_path):  # Check if the file exists
        try:
            return np.asarray(Image.open(file_path).resize(img_size)) / 255
        except Exception as e:
            print(f"Error loading image {file_path}: {e}")
            return None
    else:
        print(f"Image file not found: {file_path}")
        return None

# Train the model
history = model.fit(
    x_train, y_train,
    epochs=200,
    batch_size=8,
    validation_data=(x_test, y_test),
    callbacks=[early_stopping, reduce_lr]
)


Epoch 1/200
280/280 ━━━━━━━━━━━━━━━━━━━━ 126s 426ms/step - accuracy: 0.1740 - loss: 2.3042 - val_accuracy: 0.1306 - val_loss: 2.2806 - learning_rate: 1.0000e-04
Epoch 2/200
280/280 ━━━━━━━━━━━━━━━━━━━━ 114s 408ms/step - accuracy: 0.2046 - loss: 2.1981 - val_accuracy: 0.2165 - val_loss: 1.9265 - learning_rate: 1.0000e-04
Epoch 3/200
280/280 ━━━━━━━━━━━━━━━━━━━━ 114s 406ms/step - accuracy: 0.2553 - loss: 1.9837 - val_accuracy: 0.3524 - val_loss: 1.6041 - learning_rate: 1.0000e-04
Epoch 4/200
280/280 ━━━━━━━━━━━━━━━━━━━━ 114s 406ms/step - accuracy: 0.2648 - loss: 1.9090 - val_accuracy: 0.4401 - val_loss: 1.4971 - learning_rate: 1.0000e-04
Epoch 5/200
280/280 ━━━━━━━━━━━━━━━━━━━━ 114s 406ms/step - accuracy: 0.3050 - loss: 1.8287 - val_accuracy: 0.4401 - val_loss: 1.4163 - learning_rate: 1.0000e-04
Epoch 6/200
280/280 ━━━━━━━━━━━━━━━━━━━━ 115s 411ms/step - accuracy: 0.3236 - loss: 1.8052 - val_accuracy: 0.4723 - val_loss: 1.3562 - learning_rate: 1.0000e-04
Epoch 7/200
280/280 ━━━━━━━━━━━━━━

In [1]:
model.compile(Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])



NameError: name 'model' is not defined

In [ ]:
# Define the ReduceLROnPlateau callback to adjust the learning rate during training
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)

# Define the EarlyStopping callback to stop training if validation loss stops improving
early_stopping = EarlyStopping(patience=5)

# Train the model
history = model.fit(
    x_train, y_train,
    epochs=200,
    batch_size=8,
    validation_data=(x_test, y_test),
    callbacks=[early_stopping, reduce_lr]
)


In [ ]:
# Evaluate model
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print('Test Accuracy:', test_accuracy)
print('Test Loss:', test_loss)


In [ ]:
# Plot training history
tr_acc = history.history['accuracy']
tr_loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']

epochs = range(1, len(tr_acc) + 1)

plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs, tr_loss, 'r', label='Train Loss')
plt.plot(epochs, val_loss, 'g', label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, tr_acc, 'r', label='Train Accuracy')
plt.plot(epochs, val_acc, 'g', label='Validation Accuracy')
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()